<a href="https://colab.research.google.com/github/jorisschellekens/borb-google-colab-examples/blob/main/using_borb_to_create_a_report_cover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ![borb logo](https://github.com/jorisschellekens/borb/raw/master/logo/borb_64.png) Using `borb` to create a (report) cover PDF

[`borb`](https://github.com/jorisschellekens/borb) is a library for reading, creating and manipulating PDF files in python. borb was created in 2020 by Joris Schellekens and is still in active development. Check out the [GitHub repository](https://github.com/jorisschellekens/borb), or the [borb website](https://borbpdf.com).

In this tutorial you'll learn how to create a cover for your PDF documents. You can use this page as the first page of a test-report, a collection of invoices, or anything else you'd like.

Let's start by installing `borb`

In [259]:
pip install borb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


With that out of the way, we can now create an empty `Document` to hold our content.

In [260]:
from borb.pdf import Document
from borb.pdf import Page
from borb.pdf import SingleColumnLayout
from borb.pdf import PageLayout

# create new Document
doc: Document = Document()

# create new Page
page: Page = Page()
doc.append_page(page)

{<borb.io.read.types.Name at 0x7fa9881e0a90>: {<borb.io.read.types.Name at 0x7fa986847750>: Decimal('0'),
  <borb.io.read.types.Name at 0x7fa9881e0610>: {<borb.io.read.types.Name at 0x7fa986b50150>: {<borb.io.read.types.Name at 0x7fa9880b4b10>: {<borb.io.read.types.Name at 0x7fa986bf4a90>: Decimal('1'),
     <borb.io.read.types.Name at 0x7fa9880b4310>: [{<borb.io.read.types.Name at 0x7fa987514810>: [Decimal('0'),
        Decimal('0'),
        Decimal('595'),
        Decimal('842')],
       <borb.io.read.types.Name at 0x7fa98750b990>: {...},
       <borb.io.read.types.Name at 0x7fa987514dd0>: <borb.io.read.types.Name at 0x7fa9875143d0>}],
     <borb.io.read.types.Name at 0x7fa987fbd8d0>: <borb.io.read.types.Name at 0x7fa986befc50>},
    <borb.io.read.types.Name at 0x7fa9880b43d0>: <borb.io.read.types.Name at 0x7fa986d0ec50>}}}}

This is the tricky part. We're going to add some geometric artwork to our PDF to really spruce things up. The following code-block does just that. It adds a large triangle to the PDF, and then some smaller accent shapes. Don't worry if you don't really understand the code at first glance. Just read it a couple of times. It should hopefully become clear.

In [261]:
from decimal import Decimal
from borb.pdf import Shape
from borb.pdf import HexColor
from borb.pdf.canvas.geometry.rectangle import Rectangle
from borb.pdf import Alignment
import typing

# draw shape 1
W: Decimal = Decimal(595)           # width of paper
H: Decimal = Decimal(842)           # height of paper
W70: Decimal = Decimal(0.70) * W    # width of our large triangle
H87: Decimal = Decimal(0.87) * H    # height of our large triangle
Shape(points=[(0, H - H87), (0, H), (W70, H)], 
      fill_color=HexColor("273e4c"), 
      stroke_color=HexColor("273e4c")).layout(page, Rectangle(0, 0 , W, H))

# define a helper function
# this function returns the y-coordinate for every x-coordinate on the longest side of the triangle
# this is useful because all our other shapes will have a few coordinates in common with this side
# being able to easily calculate a point on this side of the triangle is really going to help us out
f: typing.Callable[[Decimal], Decimal] = lambda x: Decimal(109.46) + Decimal(1.7588) * x

# shape 2
ZERO: Decimal = Decimal(0)
W20: Decimal = Decimal(0.20) * W70
W40: Decimal = Decimal(0.40) * W70
W60: Decimal = Decimal(0.60) * W70
Shape(points=[(W20, f(W20)), (W40, f(W40)), (W20, f(W60)), (ZERO, f(W40))], 
      fill_color=HexColor("20dbcd"), 
      stroke_color=HexColor("20dbcd"),
      vertical_alignment=Alignment.BOTTOM).layout(page, Rectangle(ZERO, f(W20) , W, H))

# shape 3
Shape(points=[(W40, f(W40)), (W60, f(W60)), (W20, f(W60))], 
      fill_color=HexColor("2da18b"), 
      stroke_color=HexColor("2da18b"),
      vertical_alignment=Alignment.BOTTOM).layout(page, Rectangle(W20, f(W40) , W, H))

# shape 4
W80: Decimal = Decimal(0.8) * W70
Shape(points=[(W60, f(W60)), (W60 + W20*2, f(W60)), (W80 + W20*2, f(W80)), (W80, f(W80))], 
      fill_color=HexColor("20dbcd"), 
      stroke_color=HexColor("20dbcd"),
      vertical_alignment=Alignment.BOTTOM).layout(page, Rectangle(W60, f(W60) , W, H))

# shape 5
Shape(points=[(W80, f(W80)), (W80 + W20*2, f(W80)), (W + W20*2, f(W)), (W, f(W))], 
      fill_color=HexColor("00ffff"), 
      stroke_color=HexColor("00ffff"),
      vertical_alignment=Alignment.BOTTOM).layout(page, Rectangle(W80, f(W80) , W, H))

That was the hard part :-)  
I promise the next part is considerably more straightforward.  
We're just going to add some text to our `Document`.

In [262]:
from borb.pdf import Paragraph

# first paragraph
M06: Decimal = Decimal(0.06) * W
p0: Paragraph = Paragraph("PLACE TEXT", font_size=Decimal(20), font_color=HexColor("ffffff"))
p0.layout(page, Rectangle(M06, 0, W, H - M06))

# second paragraph
y: Decimal = p0.get_bounding_box().get_y() - Decimal(2)
p1: Paragraph = Paragraph("HEADLINE", font="Helvetica-bold", font_size=Decimal(30), font_color=HexColor("d3d9da"))
p1.layout(page, Rectangle(M06, 0, W, y))

# third paragraph
y: Decimal = p1.get_bounding_box().get_y() - Decimal(8)
p2: Paragraph = Paragraph("Lorem ipsum is simply dummy text of the printing and typesetting industry", font="Helvetica", 
                          font_size=Decimal(12), 
                          font_color=HexColor("ffffff"))
p2.layout(page, Rectangle(M06, 0, W40, y))


That's all the titles and headings taken care of.  
Now we'll add the main body of text. Again we're going to use absolute positioning. We have to (since our `Page` has a very uncommon layout.)

In [263]:
# fourth paragraph
p3: Paragraph = Paragraph("Lorem ipsum dolor sit amet", font="Helvetica-Bold", 
                          font_size=Decimal(20), 
                          padding_left=Decimal(20),
                          padding_right=Decimal(20),
                          font_color=HexColor("273e4c"))
p3.layout(page, Rectangle(W*Decimal(0.5), 
                          0, 
                          W * Decimal(0.5) - M06, 
                          H * Decimal(0.56)))

# main body of text
p4: Paragraph = Paragraph("""
                          Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. 
                          Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. 
                          Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. 
                          """, 
                          font_size=Decimal(12), 
                          padding_left=Decimal(20),
                          padding_right=Decimal(20),
                          text_alignment=Alignment.JUSTIFIED,
                          font_color=HexColor("273e4c"))
p4.layout(page, Rectangle(W*Decimal(0.5), 
                          0, 
                          W * Decimal(0.5) - M06, 
                          p3.get_bounding_box().get_y() - Decimal(20)))

p5: Paragraph = Paragraph("""
                          Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. 
                          Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. 
                          Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. 
                          """, 
                          font_size=Decimal(12), 
                          padding_left=Decimal(20),
                          padding_right=Decimal(20),
                          text_alignment=Alignment.JUSTIFIED,
                          font_color=HexColor("273e4c"))
p5.layout(page, Rectangle(W*Decimal(0.5), 
                          0, 
                          W * Decimal(0.5) - M06, 
                          p4.get_bounding_box().get_y() - Decimal(20)))

Finally, we can store the `Document`.

In [264]:
from borb.pdf import PDF

with open("output.pdf", "wb") as pdf_file_handle:
  PDF.dumps(pdf_file_handle, doc)

Great job!  
Go check out your PDF.